In [ ]:
%%sh
pygmentize train.R

In [ ]:
%%sh
cat Dockerfile

In [ ]:
%%sh
export REGION=eu-west-1
export ACCOUNT_ID=`aws sts get-caller-identity --query Account --output text`
docker build -t r-custom:estimator -f Dockerfile .
export IMAGE_ID=`docker images -q r-custom:estimator`
docker tag $IMAGE_ID $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/r-custom:estimator
aws ecr get-login-password | docker login --username AWS --password-stdin $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/r-custom:estimator
docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/r-custom:estimator

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator

session = sagemaker.Session()
account_id = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name

training = 'file://.'
output = 'file://.'

role = sagemaker.get_execution_role()
sk = Estimator(
             image_name=account_id+'.dkr.ecr.'+region+'eu-west-1.amazonaws.com/r-custom:estimator',
             role=role,
             train_instance_count=1, 
             train_instance_type='local',
             output_path=output,
             hyperparameters={
                  'normalize': False
             }
)

sk.fit({'training':training})

In [ ]:
%%sh
tar tvfz model.tar.gz